In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import max_error, mean_squared_error, explained_variance_score, mean_absolute_error, mean_squared_log_error, median_absolute_error, r2_score, mean_poisson_deviance, mean_gamma_deviance
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm_notebook
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures
from hyperopt import hp, fmin, STATUS_OK, tpe, Trials

In [ ]:
def set_settings():
    tf.enable_v2_behavior()
    sns.reset_defaults()
    sns.set_context(context='talk',font_scale=0.7)
    tfd = tfp.distributions

In [ ]:
def mount():
    drive.mount('/content/drive')

In [ ]:
def check_gpu_device():
    if tf.test.gpu_device_name() != '/device:GPU:0':
        print('WARNING: GPU device not found.')
    else:
        print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

In [ ]:
def prepare():
    set_settings()
    mount()
    check_gpu_device()

In [ ]:
def load():
    data = pd.read_csv('drive/MyDrive/rcsed_z_less_03.csv')
    masses = pd.read_csv('drive/MyDrive/rcsed_z_less_03_logMstar_gal.csv')
    return data, masses

In [ ]:
def prepare_data(data, masses):
    mask = ~masses['logMstar_gal'].isna()
    y = masses[mask]
    y = y.reset_index()
    y = y.drop(columns=['index'])
    data = data.drop(columns=['Unnamed: 0'])
    X = data[mask]
    X = X.reset_index()
    X = X.drop(columns=['index'])
    X = X.fillna(X.mean())
    data['g_r'] = (data['g'] + data['kcorr_g']) - (data['r'] + data['kcorr_r'])
    X['g_r'] = (X['g'] + X['kcorr_g']) - (X['r'] + X['kcorr_r'])
    return data, X, y, mask

In [ ]:
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    return X_train, X_test, y_train, y_test

In [ ]:
def scale_data(X_train, X_test):
    scal = preprocessing.RobustScaler()
    X_train = scal.fit_transform(X_train)
    X_test = scal.transform(X_test)
    return X_train, X_test

In [ ]:
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
  n = kernel_size + bias_size
  c = np.log(np.expm1(1.))
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(2 * n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t[..., :n],
                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
          reinterpreted_batch_ndims=1)),
  ])


def prior_trainable(kernel_size, bias_size=0, dtype=None):
  n = kernel_size + bias_size
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t, scale=1),
          reinterpreted_batch_ndims=1)),
  ])

In [ ]:
class RBFKernelFn(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super(RBFKernelFn, self).__init__(**kwargs)
    dtype = kwargs.get('dtype', None)

    self._amplitude = self.add_variable(
            initializer=tf.constant_initializer(0),
            dtype=dtype,
            name='amplitude')

    self._length_scale = self.add_variable(
            initializer=tf.constant_initializer(0),
            dtype=dtype,
            name='length_scale')

  def call(self, x):
    # Never called -- this is just a layer so it can hold variables
    # in a way Keras understands.
    return x

  @property
  def kernel(self):
    return tfp.math.psd_kernels.ExponentiatedQuadratic(
      amplitude=tf.nn.softplus(0.1 * self._amplitude),
      length_scale=tf.nn.softplus(5. * self._length_scale)
    )

In [ ]:
def var_model(X_train, y_train):
    tf.keras.backend.set_floatx('float64')
# Build model.
    num_inducing_points = 100
    simple_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=[56]),
        tf.keras.layers.Dense(256, kernel_initializer='ones', use_bias=False, activation='relu'),
        #tf.keras.layers.Activation('softmax'),

        tfp.layers.VariationalGaussianProcess(
            num_inducing_points=num_inducing_points,
            kernel_provider=RBFKernelFn(),
            event_shape=[1],
            unconstrained_observation_noise_variance_initializer=(
                tf.constant_initializer(np.log(np.expm1(1.)).astype('float64'))),
        ),
    ])
    batch_size = 256
    loss = lambda y, rv_y: rv_y.variational_loss(
        y, kl_weight=np.array(batch_size, 'float64') / X.shape[0])
    simple_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.008), loss=loss)

    history = simple_model.fit(X_train, y_train, batch_size=batch_size, epochs=77, verbose=2)
    return simple_model, history

In [ ]:
class EnumHyper:

    def __init__(self, estimator_func, X_train, X_test, y_train, y_test, size, random_st, params_strs, types, low, high):
        self.X_train, self.X_valid, self.y_train, self.y_valid = train_test_split(X_train, y_train, test_size=size, random_state=random_st)
        self.X_test = X_test
        self.y_test = y_test
        self.estimator_func = estimator_func
        self.params_strs = params_strs.copy()
        self.low = low.copy()
        self.high = high.copy()
        self.m = self.params_strs.shape[0]
        self.types = types


    def fn(self, params):
        for key in params:
            params[key] = self.types(params[key])
        est = self.estimator_func(**params)
        y_pred = est.fit(self.X_train, self.y_train).predict(self.X_valid)
        loss = 1 - r2_score(self.y_valid, y_pred)
        return {'loss' : loss, 'status' : STATUS_OK}


    def find_pars(self):
        space = {}
        for i in range(self.m):
            space[self.params_strs[i]] = hp.uniform(label = self.params_strs[i], low = self.low[i], high = self.high[i])
        best_params = fmin(fn=self.fn, space=space, algo=tpe.suggest, max_evals=50, verbose=False)
        return best_params


    def evaluate(self, best_params):
        for key in best_params:
            best_params[key] = self.types(best_params[key])
        est = self.estimator_func(**best_params)
        est.fit(self.X_train, self.y_train)
        y_pred_train = est.predict(self.X_train)
        y_pred_test = est.predict(self.X_test)
        sc_train = r2_score(self.y_test, y_pred_train)
        sc_test = r2_score(self.y_test, y_pred_test)
        return est, sc_train, sc_test

    def enum(self):
        best_params = self.find_pars()
        est, sc_train, sc_test = self.evaluate(best_params)
        return est, sc_train, sc_test

In [ ]:
def randForest_model(X_train, y_train):
    estimators = [150, 200]
    min_spl = [4, 5, 6]
    best_sc_first = 0
    best_est = 0
    for est in tqdm_notebook(estimators):
        rgr = RandomForestRegressor(n_estimators=est)
        rgr.fit(X_train, y_train)
        y_pred = rgr.predict(X_test)
        sc = r2_score(y_pred, y_test)
        if sc > best_sc_first:
            best_sc_first = sc
            best_est = est


    result_est = None
    best_sc_second = 0
    best_pred = None
    best_min = 0
    for cur_min in tqdm_notebook(min_spl):
        rgr = RandomForestRegressor(n_estimators=best_est, min_samples_split=cur_min)
        rgr.fit(X_train, y_train)
        y_pred = rgr.predict(X_test)
        sc = r2_score(y_pred, y_test)
        if sc > best_sc_second:
            best_sc_second = sc
            best_min = cur_min
            result_est = rgr
            best_pred = y_pred

    return result_est

In [ ]:
def randForest_model2(X_train, y_train):
    rgr = RandomForestRegressor()
    rgr.fit(X_train, y_train)
    return rgr

In [ ]:
def randForest_model3(X_train, y_train, X_test, y_test):
    enhp = EnumHyper(RandomForestRegressor, X_train, X_test, y_train, y_test, 0.33, 42, np.array(['n_estimators', 'min_samples_split']), int, [50, 4], [200, 10])
    est, sc_train, sc_test = enhp.enum()
    return est, sc_train, sc_test

In [ ]:
def lin_reg(X_train, y_train):
    rgr = LinearRegression()
    rgr.fit(X_train, y_train)
    return rgr

In [ ]:
def boosting_model(X_train, X_test, y_train, y_test):
    estimators = [50, 100, 150, 200]
    lr = [0.001, 0.01, 0.1, 0.2, 0.3]
    best_sc_first = 0
    best_est = 0
    for est in tqdm_notebook(estimators):
        rgr = XGBRegressor(n_estimators=est)
        rgr.fit(X_train, y_train)
        y_pred = rgr.predict(X_test)
        sc = r2_score(y_pred, y_test)
        if sc > best_sc_first:
            best_sc_first = sc
            best_est = est


    result_est = None
    best_sc_second = 0
    best_pred = None
    best_lr = 0
    for cur_lr in tqdm_notebook(lr):
        rgr = XGBRegressor(n_estimators=best_est, learning_rate=cur_lr)
        rgr.fit(X_train, y_train)
        y_pred = rgr.predict(X_test)
        sc = r2_score(y_pred, y_test)
        if sc > best_sc_second:
            best_sc_second = sc
            best_lr = cur_lr
            result_est = rgr
            best_pred = y_pred
    return result_est

In [ ]:
def make_predictions(model, X_train, X_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    return y_pred_train, y_pred_test

In [ ]:
def print_score(y, y_pred):
    print('max_error:',max_error(y, y_pred))
    print('mean_squared_error:',mean_squared_error(y, y_pred))
    print('explained_variance_score:', explained_variance_score(y, y_pred))
    print('mean_absolute_error:', mean_absolute_error(y, y_pred))
    print('mean_squared_log_error:', mean_squared_log_error(y, y_pred))
    print('median_absolute_error:', median_absolute_error(y, y_pred))
    print('r2_score:', r2_score(y, y_pred))
    print('mean_poisson_deviance:', mean_poisson_deviance(y, y_pred))
    print('mean_gamma_deviance:', mean_gamma_deviance(y, y_pred))
    return r2_score(y, y_pred)

In [ ]:
def scores(model, X_train, X_test, y_train, y_test):
    y_pred_train, y_pred_test = make_predictions(model, X_train, X_test)
    r2_train = print_score(y_train, y_pred_train)
    r2_test = print_score(y_test, y_pred_test)
    return r2_train, r2_test

In [ ]:
def feature_importance(model):
    importance = model.feature_importances_
    importance2 = list(filter(lambda x: x > 0.0045, importance))
    # summarize feature importance
    massiv = X.columns
    massiv2 = []
    i = 0
    for v in range(len(importance)):
      if v == 52:
        break
      if importance[v] == importance2[i]:
        print('Column: %s, Feature: %0d,  Score: %.5f' % (massiv[v], v, importance2[i]))
        massiv2.append(massiv[v])
        i += 1

    # plot feature importance
    pyplot.bar(massiv2, importance2)
    pyplot.xticks(rotation=90)
    pyplot.show()
    return massiv2

In [ ]:
def get_masses_for_unlabeled(model, data, mask):
    other_mask = 1 -  mask
    other_mask = other_mask.astype('bool')
    X_other = data[other_mask]
    X_other = X_other.reset_index()
    X_other = X_other.drop(columns=['index'])
    X_other = X_other.fillna(X_other.mean())
    res = model.predict(X_other)
    inds = np.arange(len(data))
    inds = inds[other_mask]
    my_masses = {'inds': inds, 'masses' : res}
    #df = pd.DataFrame(my_masses)
    return res

In [ ]:
def draw(model):
    yhat = model.predict(X_test)
    plt.figure(figsize=[15, 10])  # inches
    plt.plot(X, y, 'b.', label='observed', markersize=2);

    m = yhat.mean()
    s = np.std(yhat)

    plt.plot(X_test, yhat, 'r', linewidth=0.1, label='mean');
    plt.plot(X_test, yhat + 2 * s, 'g', linewidth=0.1, label=r'mean + 2 stddev');
    plt.plot(X_test, yhat - 2 * s, 'g', linewidth=0.1, label=r'mean - 2 stddev');



    ax=plt.gca();
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    ax.spines['left'].set_position(('data', 0))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    #ax.spines['left'].set_smart_bounds(True)
    #ax.spines['bottom'].set_smart_bounds(True)
    plt.legend(loc='center left', fancybox=True, framealpha=0., bbox_to_anchor=(1.05, 0.5), fontsize=5)
    plt.show()

In [ ]:
def make_all_hist(model, y_train, y_test, data, mask):
    others_y = get_masses_for_unlabeled(model, data, mask)
    sns.distplot(y_train, label='y_train')
    sns.distplot(y_test, label='y_test')
    sns.distplot(others_y, label='predicted_masses')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
def save_csv(df):
    compression_opts = dict(method='zip',
                        archive_name='out.csv')

    df.to_csv('out.zip', index=False,
          compression=compression_opts)

In [ ]:
prepare()

In [ ]:
data, masses = load()

In [ ]:
data

In [ ]:
data, X, y, mask = prepare_data(data, masses)

In [ ]:
X_train, X_test, y_train, y_test = split_data(X, y)

In [ ]:
X_train, X_test = scale_data(X_train, X_test)

In [ ]:
negloglik = lambda y, p_y: -p_y.log_prob(y)

**Model 1(tf probability)**

In [ ]:
var_model, history = var_model(X_train, y_train)

In [ ]:
simple_model_train_r2, simple_model_test_r2 = scores(var_model, X_train, X_test, y_train, y_test)

**Model 2(RandForest)**

In [ ]:
random_forest_model = randForest_model2(X_train, y_train)

In [ ]:
random_forest_model_train_r2, random_forest_model_test_r2 = scores(random_forest_model, X_train, X_test, y_train, y_test)

**Model 3(Xgboost)**

In [ ]:
boost_model = boosting_model(X_train, X_test, y_train, y_test)

In [ ]:
boost_model_train_r2, boost_model_test_r2 = scores(boost_model, X_train, X_test, y_train, y_test)

**Model 4(LinearRegression)**

In [ ]:
lin_reg_model = lin_reg(X_train, y_train)

In [ ]:
lin_reg_model_train_r2, lin_reg_model_test_r2 = scores(lin_reg_model, X_train, X_test, y_train, y_test)

**Correlation**

In [ ]:
sns.heatmap(data.corr())
plt.show()

**Feature importance**

In [ ]:
most_important_features = feature_importance(random_forest_model)

**Сomparison of three models**

In [ ]:
#table
res_table = {'names' : ['variational layer model', 'random forest model', 'boosting model'], 'train' : [simple_model_train_r2, random_forest_model_train_r2, boost_model_train_r2], 'test' : [simple_model_test_r2, random_forest_model_test_r2, boost_model_test_r2]}
res_table = pd.DataFrame(res_table)
res_table

**Masses hist**

In [ ]:
make_all_hist(random_forest_model, y_train, y_test, data, mask)

In [ ]:
other_mask = 1 -  mask
other_mask = other_mask.astype('bool')
X_other = data[other_mask]
X_other = X_other.reset_index()
X_other = X_other.drop(columns=['index'])
X_other = X_other.fillna(X_other.mean())

In [ ]:
sns.distplot(X_other['redshift'], label='pred', hist=False)
sns.distplot(X_train['redshift'], label='train',hist=False)
sns.distplot(X_test['redshift'], label='test', hist=False)
plt.legend(loc='upper left')
plt.show()

In [ ]:
plt.plot(x, y, '-ok');

**Add new features made of important features + new regression**

In [ ]:
#most_important_features - names of important features
new_data = pd.DataFrame()
for i in data.columns:
  if i in most_important_features:
    new_data[i] = data[i]
new_data

In [ ]:
poly = PolynomialFeatures(2)

In [ ]:
mask = ~masses['logMstar_gal'].isna()
new_y = masses[mask]
new_y = new_y.reset_index()
new_y = new_y.drop(columns=['index'])
new_X = new_data[mask]
new_X = new_X.reset_index()
new_X = new_X.drop(columns=['index'])
new_X = new_X.fillna(new_X.mean())

In [ ]:
new_X_transf = poly.fit_transform(new_X)

In [ ]:
X_train_new, X_test_new, y_train_new, y_test_new = split_data(new_X_transf, new_y)

In [ ]:
X_train_new, X_test_new = scale_data(X_train_new, X_test_new)

In [ ]:
new_random_forest_model = boosting_model(X_train_new, X_test_new, y_train_new, y_test_new)

In [ ]:
new_random_forest_model_train_r2, new_random_forest_model_test_r2 = scores(new_random_forest_model, X_train_new, X_test_new, y_train_new, y_test_new)